Forked from  [Object Detection with YOLO blog series](https://fairyonice.github.io/tag/object-detection-using-yolov2-on-pascal-voc2012-series.html)

Notebooks were only modified as needed, vast majority of the contents are from fairyonice.github repository.

My changes covering all notebooks were:
- Use Kaggle Wheat Detection data
- Migrate to TF 2.x
- Modified Data Generator and Loss to remove tensor error
- New notebook using albumentations for image & box augmentation
- New Kaggle submission notebook




This is the fifth blog post of [Object Detection with YOLO blog series](https://fairyonice.github.io/tag/object-detection-using-yolov2-on-pascal-voc2012-series.html). This blog finally train the model using the scripts that are developed in the [previous blog posts](https://fairyonice.github.io/tag/object-detection-using-yolov2-on-pascal-voc2012-series.html). 
I will use PASCAL VOC2012 data. 
This blog assumes that the readers have read the previous blog posts - [Part 1](https://fairyonice.github.io/Part_1_Object_Detection_with_Yolo_for_VOC_2014_data_anchor_box_clustering.html), [Part 2](https://fairyonice.github.io/Part%202_Object_Detection_with_Yolo_using_VOC_2014_data_input_and_output_encoding.html), [Part 3](https://fairyonice.github.io/Part_3_Object_Detection_with_Yolo_using_VOC_2012_data_model.html), [Part 4](https://fairyonice.github.io/Part_4_Object_Detection_with_Yolo_using_VOC_2012_data_loss.html).

## Andrew Ng's YOLO lecture
- [Neural Networks - Bounding Box Predictions](https://www.youtube.com/watch?v=gKreZOUi-O0&t=0s&index=7&list=PL_IHmaMAvkVxdDOBRg2CbcJBq9SY7ZUvs)
- [C4W3L06 Intersection Over Union](https://www.youtube.com/watch?v=ANIzQ5G-XPE&t=7s)
- [C4W3L07 Nonmax Suppression](https://www.youtube.com/watch?v=VAo84c1hQX8&t=192s)
- [C4W3L08 Anchor Boxes](https://www.youtube.com/watch?v=RTlwl2bv0Tg&t=28s)
- [C4W3L09 YOLO Algorithm](https://www.youtube.com/watch?v=9s_FpMpdYW8&t=34s)

## Reference
- [You Only Look Once:Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640.pdf) 

- [YOLO9000:Better, Faster, Stronger](https://arxiv.org/pdf/1612.08242.pdf)
 
- [experiencor/keras-yolo2](https://github.com/experiencor/keras-yolo2)

## Reference in blog
- [Part 1 Object Detection using YOLOv2 on Pascal VOC2012 - anchor box clustering](https://fairyonice.github.io/Part_1_Object_Detection_with_Yolo_for_VOC_2014_data_anchor_box_clustering.html)
- [Part 2 Object Detection using YOLOv2 on Pascal VOC2012 - input and output encoding](https://fairyonice.github.io/Part%202_Object_Detection_with_Yolo_using_VOC_2014_data_input_and_output_encoding.html)
- [Part 3 Object Detection using YOLOv2 on Pascal VOC2012 - model](https://fairyonice.github.io/Part_3_Object_Detection_with_Yolo_using_VOC_2012_data_model.html)
- [Part 4 Object Detection using YOLOv2 on Pascal VOC2012 - loss](https://fairyonice.github.io/Part_4_Object_Detection_with_Yolo_using_VOC_2012_data_loss.html)
- [Part 5 Object Detection using YOLOv2 on Pascal VOC2012 - training](https://fairyonice.github.io/Part_5_Object_Detection_with_Yolo_using_VOC_2012_data_training.html)
- [Part 6 Object Detection using YOLOv2 on Pascal VOC 2012 data - inference on image](https://fairyonice.github.io/Part_6_Object_Detection_with_Yolo_using_VOC_2012_data_inference_image.html)
- [Part 7 Object Detection using YOLOv2 on Pascal VOC 2012 data - inference on video](https://fairyonice.github.io/Part_7_Object_Detection_with_Yolo_using_VOC_2012_data_inference_video.html)

## fairyonice GitHub repository 
This repository contains all the ipython notebooks in this blog series and the funcitons (See backend.py). 
- [FairyOnIce/ObjectDetectionYolo](https://github.com/FairyOnIce/ObjectDetectionYolo)

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import tensorflow as tf
print(sys.version)
%matplotlib inline


import pandas as pd
print("Pandas: ", pd.__version__)


## Define anchor box
<code>ANCHORS</code> defines the number of anchor boxes and the shape of each anchor box.
The choice of the anchor box specialization is already discussed in [Part 1 Object Detection using YOLOv2 on Pascal VOC2012 - anchor box clustering](https://fairyonice.github.io/Part_1_Object_Detection_with_Yolo_for_VOC_2014_data_anchor_box_clustering.html). 

Based on the K-means analysis in the previous blog post, I will select 4 anchor boxes of following width and height. The width and heights are rescaled in the grid cell scale (Assuming that the number of grid size is 13 by 13.) See [Part 2 Object Detection using YOLOv2 on Pascal VOC2012 - input and output encoding](https://fairyonice.github.io/Part%202_Object_Detection_with_Yolo_using_VOC_2014_data_input_and_output_encoding.html) to learn how I rescal the anchor box shapes into the grid cell scale.

Here I choose 4 anchor boxes. With 13 by 13 grids, every frame gets 4 x 13 x 13 = 676 bouding box predictions.

In [ ]:
ANCHORS = np.array([0.06960639, 0.06130531,
                    0.11246752, 0.10739992])

## Define Label vector containing 20 object classe names.

In [ ]:
LABELS = ['wheat']


## Read images and annotations into memory
Use the pre-processing code for parsing annotation at [experiencor/keras-yolo2](https://github.com/experiencor/keras-yolo2).
This <code>parse_annoation</code> function is already used in [Part 1 Object Detection using YOLOv2 on Pascal VOC2012 - anchor box clustering](https://fairyonice.github.io/Part_1_Object_Detection_with_Yolo_for_VOC_2014_data_anchor_box_clustering.html) and saved in my python script. 
This script can be downloaded at [my Github repository, FairyOnIce/ObjectDetectionYolo/backend](https://github.com/FairyOnIce/ObjectDetectionYolo/blob/master/backend.py).

In [ ]:

# For Google Collab
#ROOT_PATH="./"    ###### CHANGE FOR SPECIFIC ENVIRONMENT

# Home
ROOT_PATH = "/Users/john/Documents/Python-Working/Kaggle-global-wheat-detection/"  ###### CHANGE FOR SPECIFIC ENVIRONMENT

# Kaggle
#ROOT_PATH = "../input/global-wheat-detection/"  ###### CHANGE FOR SPECIFIC ENVIRONMENT


TRAIN_DATA_PATH = os.path.join(ROOT_PATH, "train/")
TEST_DATA_PATH = os.path.join(ROOT_PATH, "test/")
MODEL_NAME = "model-wheat.h5"

BATCH_SIZE        = 32  # 200
IMAGE_H, IMAGE_W  = 1024, 1024
GRID_H,  GRID_W   = 13 , 13
TRUE_BOX_BUFFER   = 1183 #50
BOX               = int(len(ANCHORS)/2)


In [ ]:
# Load csvTRAIN_DATA_PATH
train_raw_df = pd.read_csv(os.path.join(ROOT_PATH, 'train.csv'))
train_raw_df.head()

In [ ]:
from backend import parse_annotation
np.random.seed(10)
train_image = parse_annotation(train_raw_df, LABELS, TRAIN_DATA_PATH, use_dict_for_bboxes=False)
print("N train = {}".format(len(train_image)))

In [ ]:
import albumentations as albu
train_augmentations = albu.Compose([
                                   albu.RandomSizedBBoxSafeCrop(IMAGE_H, IMAGE_W, p=1),
                                   albu.Flip(p=0.5),
                                   albu.OneOf([
                                               albu.HueSaturationValue(),
                                               albu.RandomBrightnessContrast()
                                              ], p=1),
                                   albu.GaussNoise(p=0.25),
                                   albu.CLAHE(p=1),
                                   albu.ToGray(p=1),
                                  ], 
                                  bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

val_augmentations = albu.Compose([
                                   albu.CLAHE(p=1),
                                   albu.ToGray(p=1),
                                  ], 
                                  bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


## Instantiate batch generator object
<code>SimpleBatchGenerator</code> is discussed and used in 
[Part 2 Object Detection using YOLOv2 on Pascal VOC2012 - input and output encoding](https://fairyonice.github.io/Part%202_Object_Detection_with_Yolo_using_VOC_2014_data_input_and_output_encoding.html).
This script can be downloaded at [my Github repository, FairyOnIce/ObjectDetectionYolo/backend](https://github.com/FairyOnIce/ObjectDetectionYolo/blob/master/backend.py).

In [ ]:
from backend import SimpleBatchGenerator, ImageReaderAlbumentations

generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'LABELS'          : LABELS,
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : TRUE_BOX_BUFFER,
}


def normalize(image):
    return image / 255.

augment = ImageReaderAlbumentations(IMAGE_H, IMAGE_W, train_augmentations, norm=normalize)

train_batch_generator = SimpleBatchGenerator(train_image,
                                             generator_config,
                                             image_reader=augment,
                                             shuffle=True)

print(len(train_image))
[x_batch, y_batch, b_batch] = train_batch_generator.__getitem__(idx=0)
print(x_batch.shape, b_batch.shape, y_batch.shape)

## Define model
We define a YOLO model.
The model defenition function is already discussed in [Part 3 Object Detection using YOLOv2 on Pascal VOC2012 - model](https://fairyonice.github.io/Part_3_Object_Detection_with_Yolo_using_VOC_2012_data_model.html) and all the codes are available at [my Github](https://github.com/FairyOnIce/ObjectDetectionYolo/blob/master/backend.py).

## Loss function
We already discussed the loss function of YOLOv2 implemented by [experiencor/keras-yolo2](https://github.com/experiencor/keras-yolo2) in [Part 4 Object Detection using YOLOv2 on Pascal VOC2012 - loss](https://fairyonice.github.io/Part_4_Object_Detection_with_Yolo_using_VOC_2012_data_loss.html).
I modified the codes and the codes are available at [my Github](https://github.com/FairyOnIce/ObjectDetectionYolo/blob/master/backend.py).

In [ ]:

BATCH_SIZE        = 32  # 32 200
IMAGE_H, IMAGE_W  = 416, 416
GRID_H,  GRID_W   = 13 , 13
TRUE_BOX_BUFFER   = 1183 #50
BOX               = int(len(ANCHORS)/2)


from backend import custom_loss_core 
#help(custom_loss_core)

LAMBDA_NO_OBJECT = 1.0
LAMBDA_OBJECT    = 5.0
LAMBDA_COORD     = 1.0
LAMBDA_CLASS     = 1.0

def custom_loss(y_true, y_pred, true_boxes):
    loss = custom_loss_core(y_true,
                     y_pred,
                     true_boxes,
                     GRID_W,
                     GRID_H,
                     BATCH_SIZE,
                     ANCHORS,
                     LAMBDA_COORD,
                     LAMBDA_CLASS,
                     LAMBDA_NO_OBJECT,
                     LAMBDA_OBJECT)
    #print("**** ", loss)
    return loss

print(custom_loss(y_batch, y_batch, b_batch))

In [ ]:
from backend import define_YOLOv2, set_pretrained_weight, initialize_weight

CLASS = len(LABELS)
(model, y_true, y_pred, true_boxes) = define_YOLOv2(IMAGE_H,
                                                    IMAGE_W,
                                                    GRID_H,
                                                    GRID_W,
                                                    TRUE_BOX_BUFFER,
                                                    BOX,CLASS, 
                                                    trainable=True)
#model.summary()


## Initialize the weights
The initialization of weights are already discussed in [Part 3 Object Detection using YOLOv2 on Pascal VOC2012 - model](https://fairyonice.github.io/Part_3_Object_Detection_with_Yolo_using_VOC_2012_data_model.html). 
All the codes from [Part 3](https://fairyonice.github.io/Part_3_Object_Detection_with_Yolo_using_VOC_2012_data_model.html) are stored at [my Github](https://github.com/FairyOnIce/ObjectDetectionYolo/blob/master/backend.py).

In [ ]:
path_to_weight = "./yolov2.weights"

nb_conv        = 22
model          = set_pretrained_weight(model,nb_conv, path_to_weight)
layer          = model.layers[-4] # -4 the last convolutional layer
initialize_weight(layer,sd=1/(GRID_H*GRID_W))

In [ ]:

model.add_loss(custom_loss(y_true, y_pred, true_boxes))


Notice that this custom function <code>custom_loss_core</code> depends not only on <code>y_true</code> and <code>y_pred</code> but also the various hayperparameters.
Unfortunately, Keras's loss function API does not accept any parameters except <code>y_true</code> and <code>y_pred</code>. Therefore, these hyperparameters need to be defined globaly. 
To do this, I will define a wrapper function <code>custom_loss</code>.

## Training starts here! 
Finally, we start the training here.
We only train the final 23rd layer and freeze the other weights.
This is because I am unfortunately using CPU environment.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

dir_log = "logs/"
try:
    os.makedirs(dir_log)
except:
    pass

generator_config['BATCH_SIZE'] = BATCH_SIZE

early_stop = EarlyStopping(monitor='loss', 
                           min_delta=0.001, 
                           patience=3, 
                           mode='min', 
                           verbose=1)

checkpoint = ModelCheckpoint(MODEL_NAME, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')


optimizer = Adam(lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optimizer = SGD(lr=1e-4, decay=0.0005, momentum=0.9)
#optimizer = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)

#model.compile(loss=custom_loss, optimizer=optimizer)
model.compile(loss=None, optimizer=optimizer)


In [ ]:
print(len(train_batch_generator))

In [ ]:


model.fit(train_batch_generator, 
                    #steps_per_epoch  = len(train_batch_generator),  ##
                    epochs           = 10, ##50
                    #verbose          = 1,  ##
                    #validation_data  = valid_batch,
                    #validation_steps = len(valid_batch),
                    callbacks        = [early_stop, checkpoint],  ##
                    #max_queue_size   = 3 ##
                     )